In [1]:
from utilities import *
from plot_utilities import *

In [31]:
sample_dict = pd.read_pickle('full-pastis-gray-wavelet/obs_x_dict.pickle')
all_cdfs = combine_pickles('scipy_10000') | combine_pickles('mtlb_10000')

In [82]:
sample_dict[6].size #// 10**6
ksstats_df = pd.DataFrame(columns = ['subsample_size', 'ksstat', '% error'])
stats.ks_1samp(compute_moving_metric(sample_dict[2], 10, np.mean), all_cdfs[(0.6, 3.2)])
stats.ks_1samp(sample_dict[2], all_cdfs[(0.4, 4)])

RangeIndex(start=0, stop=0, step=1)

In [105]:
def compute_moving_metric(sample, k, metric):
    n = len(sample)
    return metric(np.array_split(np.sort(sample), n//k), axis = 1)

ksstats_df = pd.DataFrame(columns = ['subsample_size', 'percent_sample', 'ksstat', 'abs_error', 'rel_error'])
x = sample_dict[8]
cdf = compute_prior_cdf(5, 0)


In [106]:
true_stat = stats.ks_1samp(x, cdf).statistic
print("True Stat:", true_stat)
i = 0
n = x.size
for k in np.logspace(2, np.ceil(np.log10(n)), int(np.ceil(np.log10(n))) - 1):
    subsample = np.sort(x[np.random.permutation(n)[:int(min(k, n))]])
    ksstat = stats.ks_1samp(subsample, cdf).statistic
    ksstats_df.loc[i] = k, round_to_sigfigs(k/n*100, 4), ksstat, round_to_sigfigs(ksstat - true_stat), round_to_sigfigs(np.abs(ksstat - true_stat)/true_stat * 100)
    i += 1
    
ksstats_df

True Stat: 0.22743338834433255


,subsample_size,percent_sample,ksstat,abs_error,rel_error
0,100.0,0.000768,0.265662,0.038229,16.809000
1,1000.0,0.007677,0.241894,0.014460,6.358100
2,10000.0,0.076770,0.228540,0.001106,0.486470
3,100000.0,0.767700,0.227708,0.000274,0.120630
4,1000000.0,7.677000,0.227390,-0.000044,0.019142
5,10000000.0,76.770000,0.227379,-0.000054,0.023952
6,100000000.0,767.700000,0.227433,0.000000,0.000000


In [103]:
stats.ks_1samp(sample_dict[6], all_cdfs[(1, 1)])

KstestResult(statistic=0.034997237265525505, pvalue=0.0, statistic_location=0.4601189090764415, statistic_sign=1)

In [ ]:
def compute_ksstat(sample, cdf, sorted_sample = True):
    '''
    Computes the KS-Test Statistic, assumes that the sample is already sorted
    '''
    if not sorted_sample:
        sample = np.sort(sample)

    if isinstance(cdf, tuple):
        r = cdf[0]
        eta = cdf[1]
        cdf = compute_prior_cdf(r, eta, 10000)
    
    n = len(sample)
    cdfvals = cdf(sample)
    dplus, dminus = (np.arange(1.0, n + 1) / n - cdfvals), (cdfvals - np.arange(0.0, n)/n)
    return np.max(np.append(dplus, dminus))

def approx_ksstat(sample, cdf):
    